# Laboratório 1 - Parte 2: Ranking usando o Modelo Vetorial (VSM)

## Aluno: Paulo Vinícius Soares
## 21/05/2018

### Introdução

Nesse laboratório vamos nos aprofundar um pouco mais nos conceitos de busca de documentos através de palavras-chave. Vamos adotar a estratégia de ordenação destes através de um ranqueamento gerado a partir de alguns algoritmos. Os algoritmos escolhidos para essa análise foram: **Representação Binária**, **TF**, **TF-IDF** e **BM25**.

Vamos aos *imports*. Vamos utilizar nossos três amigos já conhecidos: *pandas*, *nltk* e o *numpy*. Vamos também importar a biblioteca *operator* para ordenação do resultado final.

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
import math
import operator

In [2]:
DATABASE_PATH = '../database/estadao_noticias_eleicao.csv'
TEMPLATE_PATH = '../database/gabarito.csv'

A nossa base de dados é uma versão estendida da primeira parte do laboratório, um conjunto de notícias políticas coletadas no [Estadão Online](http://www.estadao.com.br/).

In [3]:
noticias_estadao = pd.read_csv(DATABASE_PATH)
noticias_estadao = noticias_estadao.replace(np.nan, '', regex=True)
gabarito = pd.read_csv(TEMPLATE_PATH)

Após importar corretamente a base de dados, podemos dar uma olhada nos novos campos do *dataframe*. Agora contamos com **subTitulo**, **url** e **timestamp** para auxiliar na construção do *ranking*. Porém, os campos utilizados para a análise são: **titulo**, **subTitulo** e **conteudo**.

In [4]:
noticias_estadao.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


### Construindo o novo índice invertido

A nova função de índice invertido deve guardar também a frequência que os termos aparecem em cada documento. Dessa forma a estratégia adotada foi a criação de um mapa de mapas, onde a chave do mapa mais externo será o **termo** e o mapa mais interno conterá o **idNoticia** juntamente com a **frequência do termo naquela notícia**.

In [5]:
def freq_termo(documento, termo):
    return len(list(filter((lambda x: x == termo), documento)))

In [6]:
def produz_tokens(df):
    inverted_index = {}
    for i, row in df.iterrows():
        
        titulo = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        subtitulo = (word.lower() for word in (nl.word_tokenize(row['subTitulo'])))
        conteudo = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        
        documento = list(titulo) + list(subtitulo) + list(conteudo)
        
        for termo in documento:
            if termo in inverted_index:
                inverted_index[termo][termo][row['idNoticia']] = freq_termo(documento, termo)
            else:
                inverted_index[termo] = {termo: {row['idNoticia']: freq_termo(documento, termo)}}
        
    return inverted_index

In [7]:
general_dict = produz_tokens(noticias_estadao)

### Construindo a representação binária

A representação binária analisa a *query* de consulta verificando se o termo aparece no documento. Caso apareça, é somado 1 ponto de *score* para aquele documento. Se a *query* possuir três termos e no documento de id 1 os três termos aparecem, a função retorna o *score* 3 para esse documento.

Essa estratégia é bastante simples e de fácil implementação, mas não funciona muito bem por desconsiderar alguns aspectos mais detalhados do documento. Os casos de empate podem ser bastante frequentes, dependendo do tamanho da query, e não há um critério para desempate.

Espaço reservado para fotos

In [8]:
def representacao_binaria(query):
    binary_dict = {}
    for termo in query:
        for docId in general_dict[termo][termo].keys():
            if docId in binary_dict:
                binary_dict[docId] += 1
            else:
                binary_dict[docId] = 1
    return binary_dict

### TF (Term Frequency)

A ideia aqui é bem parecida com a representação binária, mas ao invés de considerar 1 ponto para cada vez que o termo aparece, consideramos a frequência com que o termo aparece no documento. Caso ele apareça muito no documento, maior será sua pontuação.

A ideia é um pouco melhor do que a anterior por reduzir o empate no *ranking* dos documentos, mas pode ser facilmente enviesado pela frequência abusiva de um termo no documento. 

Espaço reservado para fotos do babado

In [9]:
def tf(query):
    tf_dict = {}
    for termo in query:
        for docId in general_dict[termo][termo].keys():
            if docId in tf_dict:
                tf_dict[docId] += general_dict[termo][termo][docId]
            else:
                tf_dict[docId] = general_dict[termo][termo][docId]
    return tf_dict

### TF-IDF (Inverse Document Frequency)

Visando melhorar esse viés da frequência abusiva dos termos no documento, o IDF é calculado baseado na relação entre o **número de documentos em que o termo aparece** e o **número total de documentos**. O logaritmo dessa relação visa suavizar a curva para casos em que a quantidade de termos é muito elevada. Dessa forma, uma quantidade *x* de termos encontrados no documento não influencia tanto assim no *score* final. Então calcula-se o **TF** e, em seguida, é multiplicado pelo **IDF** de cada documento.

Espaço reservado para fotos do babado

In [10]:
def tf_idf(query):
    M = noticias_estadao.size # Total de documentos
    tf_idf_dict = {}
    for termo in query:
        for docId in general_dict[termo][termo].keys():
            k = len(general_dict[termo][termo])
            if docId in tf_idf_dict:               
                tf_idf_dict[docId] += np.log((M+1)/k)
            else:
                tf_idf_dict[docId] = np.log((M+1)/k)
    
    tf_dict = tf(query)
    
    for docId in tf_dict.keys():
        tf_idf_dict[docId] *= tf_dict[docId]
    
    return tf_idf_dict

### BM25

O **BM25** considera todos os critérios definidos previamente: **frequência do termo no documento**, **quantidade de documentos em que o termo aparece** e a **quantidade de documentos na base de dados**. Esse é o algoritmo mais robusto por apresentar um limite superior para o *score*, além de tratar de forma mais adequada a repetição abusiva de termos.

Espaço reservado para fotos da fórmula do bm25

In [11]:
def bm25(query):
    M = noticias_estadao.size
    bm25_dict = {}
    for termo in query:
        for docId in general_dict[termo][termo].keys():
            k = len(general_dict[termo][termo])
            c = general_dict[termo][termo][docId]
            if docId in bm25_dict:               
                bm25_dict[docId] += (((k+1)*c)/(c+k))*np.log((M+1)/k)
            else:
                bm25_dict[docId] = (((k+1)*c)/(c+k))*np.log((M+1)/k)
    return bm25_dict

### Definição das funções de busca

Vamos definir uma função **busca** que encapsula as funções acima:

In [12]:
from enum import Enum

class ALGORITMO_VSM(Enum):
    BINARIO = 1
    TF = 2
    TFIDF = 3
    BM25 = 4

In [13]:
def busca(query_string, tipo_algoritmo):
    query = query_string.split()
    
    scores = {}
    
    if tipo_algoritmo is ALGORITMO_VSM.BINARIO:
        scores = representacao_binaria(query)
    elif tipo_algoritmo is ALGORITMO_VSM.TF:
        scores = tf(query)
    elif tipo_algoritmo is ALGORITMO_VSM.TFIDF:
        scores = tf_idf(query)
    elif tipo_algoritmo is ALGORITMO_VSM.BM25:
        scores = bm25(query)
    else:
        raise Exception("Algoritmo não reconhecido")
    
    sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse = True)
    
    ids = []
    for i in range(5):
        ids.append(sorted_scores[i][0])

    return ids
        

### Definindo função de medição de acertos/erros

In [14]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

### Realizando as consultas

As consultas serão realizadas considerando os seguintes termos:
    1. segundo turno;
    2. lava jato;
    3. projeto de lei;
    4. compra de voto.
    5. ministério público.

#### segundo turno

In [15]:
gabarito.head()

,str_busca,google,busca_binaria,tf,tfidf,bm25
0,segundo turno,"[1062, 1942, 2161, 2078, 2073]","[2048, 1, 2049, 2050, 4096]","[2744, 7, 2112, 7672, 2388]","[2744, 2112, 7672, 1235, 2388]","[2744, 2112, 7672, 2388, 2178]"
1,lava jato,"[616, 164, 1734, 163, 6716]","[3, 13, 15, 27, 6177]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]","[163, 353, 2807, 127, 359]"
2,projeto de lei,"[2853, 275, 978, 7092, 3171]","[3584, 6145, 8194, 8706, 6660]","[7, 3942, 7017, 1250, 6942]","[2232, 6461, 2853, 3171, 3942]","[2232, 6461, 3171, 2853, 3170]"
3,compra de voto,"[2200, 8615, 2265, 7746, 82]","[7424, 2178, 6531, 5122, 2311]","[3942, 7017, 5129, 2047, 748]","[7343, 7293, 6791, 3942, 2047]","[7343, 7293, 6791, 7329, 8615]"
4,ministério público,"[64, 6652, 164, 6550, 8615]","[8194, 7, 4104, 8201, 4109]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]","[6798, 8018, 6244, 6965, 6550]"


In [16]:
print(busca("segundo turno", ALGORITMO_VSM.BINARIO))
print(busca("segundo turno", ALGORITMO_VSM.TF))
print(busca("segundo turno", ALGORITMO_VSM.TFIDF))
print(busca("segundo turno", ALGORITMO_VSM.BM25))

a = busca("segundo turno", ALGORITMO_VSM.BINARIO)

print(type(a))

mapk(gabarito.busca_binaria, a, k = 5)

[1, 7, 13, 26, 69]
[2744, 7, 2112, 7672, 2388]
[2744, 7, 2112, 7672, 2388]
[2744, 2112, 7672, 7, 2388]
<class 'list'>


TypeError: object of type 'int' has no len()

#### lava jato

In [ ]:
print(busca("lava jato", ALGORITMO_VSM.BINARIO))
print(busca("lava jato", ALGORITMO_VSM.TF))
print(busca("lava jato", ALGORITMO_VSM.TFIDF))
print(busca("lava jato", ALGORITMO_VSM.BM25))

#### projeto de lei

In [ ]:
print(busca("projeto de lei", ALGORITMO_VSM.BINARIO))
print(busca("projeto de lei", ALGORITMO_VSM.TF))
print(busca("projeto de lei", ALGORITMO_VSM.TFIDF))
print(busca("projeto de lei", ALGORITMO_VSM.BM25))

#### compra de voto

In [ ]:
print(busca("compra de voto", ALGORITMO_VSM.BINARIO))
print(busca("compra de voto", ALGORITMO_VSM.TF))
print(busca("compra de voto", ALGORITMO_VSM.TFIDF))
print(busca("compra de voto", ALGORITMO_VSM.BM25))

#### ministério público

In [ ]:
print(busca("ministério público", ALGORITMO_VSM.BINARIO))
print(busca("ministério público", ALGORITMO_VSM.TF))
print(busca("ministério público", ALGORITMO_VSM.TFIDF))
print(busca("ministério público", ALGORITMO_VSM.BM25))